# tensorflow-serving

Tensorflow is widely used in industrial domain, because of its wide support for different scenario, such as web, embedding system, etc.

In this notebook, we talk about tensorflow-serving which supplies web API service.

4 steps to deploy tensorflow serving:
    1. prepare protobuf model file
    2. install tensorflow serving
    3. start tensorflow serving
    4. send request by API and get prediction

In [1]:
import tensorflow as tf

## 1. prepare protobuf model file

In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers

## nb_samples
n = 800

## Generate training data
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n,1],
    mean = 0.0,stddev= 2.0) 

## create model
tf.keras.backend.clear_session()
inputs = layers.Input(shape = (2,),name ="inputs") #设置输入名字为inputs
outputs = layers.Dense(1, name = "outputs")(inputs) #设置输出名字为outputs
linear = models.Model(inputs = inputs,outputs = outputs)
linear.summary()

## training
linear.compile(optimizer="rmsprop",loss="mse",metrics=["mae"])
linear.fit(X,Y,batch_size = 8,epochs = 100)  

tf.print("w = ",linear.layers[1].kernel)
tf.print("b = ",linear.layers[1].bias)

## save model to protobuf format
export_path = "./data/linear_model/"
version = "1"       #后续可以通过版本号进行模型版本迭代与管理
linear.save(export_path+version, save_format="tf")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
outputs (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
Train on 800 samples
Epoch 1/100
800/800 [==============================] - 0s 484us/sample - loss: 97.8517 - mae: 8.2274
Epoch 2/100
800/800 [==============================] - 0s 183us/sample - loss: 86.9470 - mae: 7.7588
Epoch 3/100
800/800 [==============================] - 0s 167us/sample - loss: 76.9889 - mae: 7.2992
Epoch 4/100
800/800 [==============================] - 0s 168us/sample - loss: 67.6229 - mae: 6.8392
Epoch 5/100
800/800 [==============================] - 0s 251us/sample - loss: 59.0061 - mae: 6.3920
Epo

800/800 [==============================] - 0s 189us/sample - loss: 3.9299 - mae: 1.5789
Epoch 76/100
800/800 [==============================] - 0s 161us/sample - loss: 3.9341 - mae: 1.5781
Epoch 77/100
800/800 [==============================] - 0s 171us/sample - loss: 3.9317 - mae: 1.5793
Epoch 78/100
800/800 [==============================] - 0s 170us/sample - loss: 3.9340 - mae: 1.5797
Epoch 79/100
800/800 [==============================] - 0s 161us/sample - loss: 3.9335 - mae: 1.5791
Epoch 80/100
800/800 [==============================] - 0s 165us/sample - loss: 3.9336 - mae: 1.5790
Epoch 81/100
800/800 [==============================] - 0s 160us/sample - loss: 3.9326 - mae: 1.5786
Epoch 82/100
800/800 [==============================] - 0s 161us/sample - loss: 3.9325 - mae: 1.5793
Epoch 83/100
800/800 [==============================] - 0s 158us/sample - loss: 3.9336 - mae: 1.5798
Epoch 84/100
800/800 [==============================] - 0s 164us/sample - loss: 3.9323 - mae: 1.5787
Epo

In [3]:
!ls {export_path+version}

assets         saved_model.pb variables


In [4]:
!saved_model_cli show --dir {export_path+str(version)} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: serving_default_inputs:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['outputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
    Option #1

## 2. install tensorflow serving

Install docker

Run cmd:

docker pull tensorflow/serving

## 3. Start tensorflow serving

!docker run -t --rm -p 8501:8501 \
    -v "/Users/.../data/linear_model/" \
    -e MODEL_NAME=linear_model \
    tensorflow/serving & >server.log 2>&1

## 4. Send request

!curl -d '{"instances": [1.0, 2.0, 5.0]}' \
    -X POST http://localhost:8501/v1/models/linear_model:predict